In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.6 MB/s eta 0:00:00


In [16]:
import torch
import pandas as pd
import plotly.graph_objs as go
from torch_geometric.data import Data

# Load the dataset
file_path = '/content/drive/MyDrive/Iqram Sir/Final_data_train.csv'
df = pd.read_csv(file_path)

# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()

# List of joints in the order they appear in the DataFrame
joints = [
    'FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
    'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
    'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
    'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
    'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
    'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT'
]

# Function to extract node features and create a Data object
def create_data_object_vis(row):
    node_features = []
    node_positions = []  # Store (x, y, z) positions separately for Plotly visualization

    for joint in joints:
        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])
        node_positions.append((x, y, z))  # Save position for each joint

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['frailty_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data, node_positions

# Function to plot the graph using Plotly
def plot_graph(node_positions):
    # Extract x, y, z coordinates from node_positions
    x_coords = [pos[0] for pos in node_positions]
    y_coords = [pos[1] for pos in node_positions]
    z_coords = [pos[2] for pos in node_positions]

    # Create edges for Plotly
    edge_x = []
    edge_y = []
    edge_z = []
    for edge in edges.t().tolist():
        x0, y0, z0 = node_positions[edge[0]]
        x1, y1, z1 = node_positions[edge[1]]
        edge_x.extend([x0, x1, None])
        edge_y.extend([y0, y1, None])
        edge_z.extend([z0, z1, None])

    # Plot the edges
    edge_trace = go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='black', width=2),
        hoverinfo='none'
    )

    # Plot the nodes with joint names
    node_trace = go.Scatter3d(
        x=x_coords, y=y_coords, z=z_coords,
        mode='markers+text',
        marker=dict(size=6, color='blue'),
        text=joints,  # Use the joint names as text labels
        hoverinfo='text'
    )

    # Create the figure with gridlines and joint names
    fig = go.Figure(data=[edge_trace, node_trace],
                    layout=go.Layout(
                        title='3D  Visualization',
                        showlegend=False,
                        scene=dict(
                            xaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            yaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                            zaxis=dict(
                                showbackground=True,
                                backgroundcolor="rgb(230, 230, 230)",
                                gridcolor="rgb(200, 200, 200)",
                                showgrid=True,
                                zerolinecolor="rgb(200, 200, 200)",
                            ),
                        ),
                        margin=dict(l=0, r=0, b=0, t=40),
                    ))

    fig.show()

# Example usage with a single row (e.g., the first row in your DataFrame)
single_row = df.iloc[0]
data, node_positions = create_data_object_vis(single_row)

# Plot the graph using Plotly
plot_graph(node_positions)


In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
import pandas as pd
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np

# Load the CSV file
file_path = '/content/drive/MyDrive/Iqram Sir/shuffled_data_train.csv'
df = pd.read_csv(file_path)


In [10]:
# Define anatomical connections as edges
edges = torch.tensor([
    [0, 2],  # FOOT_RIGHT to ANKLE_RIGHT
    [1, 3],  # FOOT_LEFT to ANKLE_LEFT
    [2, 4],  # ANKLE_RIGHT to KNEE_RIGHT
    [3, 5],  # ANKLE_LEFT to KNEE_LEFT
    [4, 6],  # KNEE_RIGHT to HIP_RIGHT
    [5, 7],  # KNEE_LEFT to HIP_LEFT
    [6, 8],  # HIP_RIGHT to PELVIS
    [7, 8],  # HIP_LEFT to PELVIS
    [8, 9],  # PELVIS to SPINE_NAVAL
    [9, 10], # SPINE_NAVAL to SPINE_CHEST
    [10, 11],# SPINE_CHEST to CLAVICLE_RIGHT
    [10, 12],# SPINE_CHEST to CLAVICLE_LEFT
    [11, 13],# CLAVICLE_RIGHT to SHOULDER_RIGHT
    [12, 14],# CLAVICLE_LEFT to SHOULDER_LEFT
    [13, 15],# SHOULDER_RIGHT to ELBOW_RIGHT
    [14, 16],# SHOULDER_LEFT to ELBOW_LEFT
    [15, 17],# ELBOW_RIGHT to WRIST_RIGHT
    [16, 18],# ELBOW_LEFT to WRIST_LEFT
    [17, 19],# WRIST_RIGHT to HAND_RIGHT
    [18, 20],# WRIST_LEFT to HAND_LEFT
    [19, 21],# HAND_RIGHT to HANDTIP_RIGHT
    [20, 22],# HAND_LEFT to HANDTIP_LEFT
    [21, 23],# HANDTIP_RIGHT to THUMB_RIGHT
    [22, 24],# HANDTIP_LEFT to THUMB_LEFT
    [10, 25],# SPINE_CHEST to NECK
    [25, 26],# NECK to HEAD
]).t().contiguous()


In [11]:
# Function to extract node features and create a Data object
def create_data_object(row):
    node_features = []

    for joint in ['FOOT_RIGHT', 'FOOT_LEFT', 'ANKLE_RIGHT', 'ANKLE_LEFT', 'KNEE_RIGHT', 'KNEE_LEFT',
                  'HIP_RIGHT', 'HIP_LEFT', 'PELVIS', 'SPINE_NAVAL', 'SPINE_CHEST',
                  'CLAVICLE_RIGHT', 'CLAVICLE_LEFT', 'SHOULDER_RIGHT', 'SHOULDER_LEFT',
                  'ELBOW_RIGHT', 'ELBOW_LEFT', 'WRIST_RIGHT', 'WRIST_LEFT', 'HAND_RIGHT',
                  'HAND_LEFT', 'HANDTIP_RIGHT', 'HANDTIP_LEFT', 'THUMB_RIGHT', 'THUMB_LEFT',
                  'NECK', 'HEAD', 'NOSE', 'EYE_LEFT', 'EAR_LEFT', 'EYE_RIGHT', 'EAR_RIGHT']:

        x = row[f'{joint}_X']
        y = row[f'{joint}_Y']
        z = row[f'{joint}_Z']
        node_features.append([x, y, z])

    node_features = torch.tensor(node_features, dtype=torch.float)

    # Extract label (assuming 'frailty' is the target column)
    label = torch.tensor([row['frailty_class']], dtype=torch.float)

    # Create the Data object
    data = Data(x=node_features, edge_index=edges, y=label)

    return data


In [12]:
# Apply the function to each row in the dataframe
data_list = [create_data_object(row) for index, row in df.iterrows()]

# Now data_list contains the graph data for each row in the dataset


In [13]:
# Example ST-GCN model definition
"""class STGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(STGCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.relu = nn.ReLU()
        self.fc = nn.Linear(out_channels, 1)  # Assuming binary classification

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = self.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.mean(x, dim=0)  # Global mean pooling
        out = self.fc(x)
        return torch.sigmoid(out)


"""
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool, BatchNorm, global_max_pool

class STGCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(STGCN, self).__init__()

        # First graph convolution layer
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.bn1 = BatchNorm(hidden_channels)  # Batch normalization after the first GCN layer

        # Second graph convolution layer
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.bn2 = BatchNorm(hidden_channels)  # Batch normalization after the second GCN layer

        # Third graph convolution layer
        self.conv3 = GCNConv(hidden_channels, out_channels)
        self.bn3 = BatchNorm(out_channels)  # Batch normalization after the third GCN layer

        # Fully connected layer
        self.fc = nn.Linear(out_channels, 1)  # Assuming binary classification

        # Dropout layer for regularization
        self.dropout = nn.Dropout(p=0.5)

        # Activation function
        self.relu = nn.ReLU()

    def forward(self, x, edge_index, batch=None):
        # First layer
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = self.relu(x)

        # Second layer
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = self.relu(x)

        # Third layer
        x = self.conv3(x, edge_index)
        x = self.bn3(x)
        x = self.relu(x)

        # Global mean pooling or max pooling
        if batch is not None:
            x = global_mean_pool(x, batch)  # Or use global_max_pool(x, batch)
        else:
            x = torch.mean(x, dim=0)

        # Dropout for regularization
        x = self.dropout(x)

        # Fully connected layer and output
        out = self.fc(x)
        return torch.sigmoid(out)

# Example usage:
# model = STGCN(in_channels=3, hidden_channels=64, out_channels=128)



In [14]:

if torch.cuda.is_available():
    print("CUDA is available. Using GPU for training.")
    device = torch.device('cuda')
else:
    print("CUDA is not available. Using CPU for training.")
    device = torch.device('cpu')


# Hyperparameters
in_channels = 3  # Number of features per node (e.g., x, y, z)
hidden_channels = 8
out_channels = 4
learning_rate = 0.01
num_epochs = 10


# Example model, optimizer, and loss
model = STGCN(in_channels, hidden_channels, out_channels).to(device)


optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()

# Function to calculate accuracy
def calculate_accuracy(output, labels):
    preds = (output > 0.5).float()  # Convert probabilities to binary predictions
    correct = (preds == labels).sum().item()
    accuracy = correct / len(labels)
    return accuracy

# Example training loop with progress bar
for epoch in tqdm(range(num_epochs)):
    model.train()
    optimizer.zero_grad()

    # Training loop
    losses = []
    accuracies = []
    for data in data_list:
        data = data.to(device)
        output = model(data.x, data.edge_index)
        loss = criterion(output, data.y)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
        accuracies.append(calculate_accuracy(output, data.y))

    # Print metrics
    avg_loss = np.mean(losses)
    avg_accuracy = np.mean(accuracies)
    print(f'Epoch {epoch + 1}/{num_epochs}')
    print(f'Training Loss: {avg_loss:.4f}, Training Accuracy: {avg_accuracy:.4f}')
    print("_-----------_---------_----------_---------_----------_--------_---------")


CUDA is available. Using GPU for training.


 10%|█         | 1/10 [02:33<23:03, 153.76s/it]

Epoch 1/10
Training Loss: 1.1958, Training Accuracy: 0.5047
_-----------_---------_----------_---------_----------_--------_---------


 20%|██        | 2/10 [05:02<20:05, 150.72s/it]

Epoch 2/10
Training Loss: 1.3393, Training Accuracy: 0.5056
_-----------_---------_----------_---------_----------_--------_---------


...

Epoch 8/10
Training Loss: 1.2775, Training Accuracy: 0.5056
_-----------_---------_----------_---------_----------_--------_---------


 90%|█████████ | 9/10 [22:22<02:28, 148.69s/it]

Epoch 9/10
Training Loss: 1.2345, Training Accuracy: 0.5080
_-----------_---------_----------_---------_----------_--------_---------


100%|██████████| 10/10 [24:51<00:00, 149.15s/it]

Epoch 10/10
Training Loss: 1.0465, Training Accuracy: 0.5079
_-----------_---------_----------_---------_----------_--------_---------


In [ ]:
# Step 1: Load the testing dataset
test_file_path = '/content/drive/MyDrive/Iqram Sir/Final_data_valid.csv'  # Replace with your actual path
df_test = pd.read_csv(test_file_path)

# Step 2: Preprocess the testing data (same as training preprocessing)
# Note: Unpack the returned tuple to get the Data object
test_data_list = [create_data_object(row)[0] for index, row in df_test.iterrows()]  # Only keep the Data object

# Step 3: Evaluate the model
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Switch the model to evaluation mode
model.eval()

# Lists to store all predictions and true labels
all_preds = []
all_labels = []

# Evaluate on the test data
with torch.no_grad():
    for data in test_data_list:
        data = data.to(device)  # Move Data object to the device
        output = model(data.x, data.edge_index)
        pred = (output > 0.5).float()  # Convert probabilities to binary predictions

        all_preds.append(pred.item())
        all_labels.append(data.y.item())

# Convert lists to tensors for evaluation
all_preds = torch.tensor(all_preds)
all_labels = torch.tensor(all_labels)

# Calculate evaluation metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds)
recall = recall_score(all_labels, all_preds)
f1 = f1_score(all_labels, all_preds)
conf_matrix = confusion_matrix(all_labels, all_preds)

# Print evaluation metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')
print('Confusion Matrix:')
print(conf_matrix)
